In [2]:
!pip install pandas scikit-learn

In [14]:
# Importação de bibliotecas necessárias
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import resample

In [16]:
def process_and_train(dataset_path, model, model_name):
    '''
    Função para processar o conjunto de dados, balancear as classes e treinar o modelo.
    
    Parâmetros:
    - dataset_path (str): Caminho para o arquivo CSV do dataset.
    - model: Instância do modelo a ser treinado.
    - model_name (str): Nome do modelo, usado para exibição nos resultados.
    '''
    print(f"Processando o arquivo: {dataset_path}")
    
    # Carregar o dataset
    df = pd.read_csv(dataset_path)
    
    # Dividir os dados em treino e teste
    x_train, x_test, y_train, y_test = train_test_split(
        df['text'], df['class'], test_size=0.2, stratify=df['class'], random_state=42
    )
    
    # Verificar a distribuição de classes no conjunto de treino
    print("Distribuição de classes no conjunto de treino antes do balanceamento:")
    print(y_train.value_counts())
    
    # Balancear os dados de treino (oversampling das classes menores)
    df_train = pd.DataFrame({'text': x_train, 'class': y_train})
    majority_class = df_train['class'].value_counts().idxmax()
    df_majority = df_train[df_train['class'] == majority_class]
    df_minority = df_train[df_train['class'] != majority_class]
    
    df_minority_upsampled = resample(df_minority,
                                     replace=True,
                                     n_samples=len(df_majority),
                                     random_state=42)
    df_balanced = pd.concat([df_majority, df_minority_upsampled])
    
    # Atualizar os dados de treino balanceados
    x_train = df_balanced['text']
    y_train = df_balanced['class']
    
    # Vetorização e transformação TF-IDF
    vectorizer = CountVectorizer()
    x_train_vec = vectorizer.fit_transform(x_train)
    x_test_vec = vectorizer.transform(x_test)
    
    tfidf = TfidfTransformer()
    x_train_tfidf = tfidf.fit_transform(x_train_vec)
    x_test_tfidf = tfidf.transform(x_test_vec)
    
    # Treinar o modelo
    model.fit(x_train_tfidf, y_train)
    
    # Fazer previsões e avaliar o modelo
    y_pred = model.predict(x_test_tfidf)
    print(f"\nResultados para o modelo {model_name} e arquivo {dataset_path}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Evitar métricas indefinidas
    print(f"F1-score (macro): {f1_score(y_test, y_pred, average='macro'):.2f}")

In [18]:
# Executar para Naive Bayes
nb_model = MultinomialNB()
process_and_train("C:/Users/tagsa/Downloads/classic4.csv", nb_model, "Naive Bayes")
process_and_train("C:/Users/tagsa/Downloads/Industry Sector.csv", nb_model, "Naive Bayes")

Processando o arquivo: C:/Users/tagsa/Downloads/classic4.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
cacm    2563
cisi    1168
cran    1118
med      827
Name: count, dtype: int64

Resultados para o modelo Naive Bayes e arquivo C:/Users/tagsa/Downloads/classic4.csv:
              precision    recall  f1-score   support

        cacm       0.87      0.99      0.93       641
        cisi       0.96      0.79      0.86       292
        cran       0.97      0.96      0.97       280
         med       1.00      0.84      0.92       206

    accuracy                           0.92      1419
   macro avg       0.95      0.90      0.92      1419
weighted avg       0.93      0.92      0.92      1419

F1-score (macro): 0.92
Processando o arquivo: C:/Users/tagsa/Downloads/Industry Sector.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
services          2076
consumer          1238
financial          767
basic              757
capi

In [20]:
# Executar para Logistic Regression
lr_model = LogisticRegression(max_iter=200, solver='liblinear')
process_and_train("C:/Users/tagsa/Downloads/classic4.csv", lr_model, "Logistic Regression")
process_and_train("C:/Users/tagsa/Downloads/Industry Sector.csv", lr_model, "Logistic Regression")

Processando o arquivo: C:/Users/tagsa/Downloads/classic4.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
cacm    2563
cisi    1168
cran    1118
med      827
Name: count, dtype: int64

Resultados para o modelo Logistic Regression e arquivo C:/Users/tagsa/Downloads/classic4.csv:
              precision    recall  f1-score   support

        cacm       0.91      0.98      0.95       641
        cisi       0.95      0.87      0.91       292
        cran       0.99      0.95      0.97       280
         med       1.00      0.93      0.96       206

    accuracy                           0.95      1419
   macro avg       0.96      0.93      0.95      1419
weighted avg       0.95      0.95      0.95      1419

F1-score (macro): 0.95
Processando o arquivo: C:/Users/tagsa/Downloads/Industry Sector.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
services          2076
consumer          1238
financial          767
basic              

In [22]:
# Executar para Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
process_and_train("C:/Users/tagsa/Downloads/classic4.csv", rf_model, "Random Forest")
process_and_train("C:/Users/tagsa/Downloads/Industry Sector.csv", rf_model, "Random Forest")

Processando o arquivo: C:/Users/tagsa/Downloads/classic4.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
cacm    2563
cisi    1168
cran    1118
med      827
Name: count, dtype: int64

Resultados para o modelo Random Forest e arquivo C:/Users/tagsa/Downloads/classic4.csv:
              precision    recall  f1-score   support

        cacm       0.79      0.99      0.88       641
        cisi       0.97      0.69      0.80       292
        cran       0.99      0.89      0.94       280
         med       1.00      0.76      0.86       206

    accuracy                           0.87      1419
   macro avg       0.94      0.83      0.87      1419
weighted avg       0.90      0.87      0.87      1419

F1-score (macro): 0.87
Processando o arquivo: C:/Users/tagsa/Downloads/Industry Sector.csv
Distribuição de classes no conjunto de treino antes do balanceamento:
class
services          2076
consumer          1238
financial          767
basic              757
ca